# Modeling Tennis Matches via Absorbing Markov Chains
###Siddharth Sharma, MATH 104, Prof. Gene Kim

Modeled points and then games in a nested absorbing markov chain and found their stable states for large N. Then simulated sets and matches by having pairs of service games till someone reached a winning outcome (6 games for a set)


In [124]:
import numpy as np
import random

#temperature = random number between -0.025 & 0.025
tmp = random.randint(-15, 15) / 1000

p_fserve = 0.675 + tmp
p_sserve = 0.30 - tmp
p_df = 0.025
p_fh = 0.25 - tmp
p_bh = 0.15 + tmp
p_w = 0.35 + tmp
p_e = 0.25 - tmp

fh_boost = 0.05
fserve_boost = 0.05
# 1st serves and forehands are more optimal (offensive/dominant shots)
# backhands + second serves are more defensive (and more error-prone)

# states: 1st serve, 2nd serve, double fault, forehand, backhand, winner, error

init_state = np.array([p_fserve, p_sserve, p_df, 0, 0, 0, 0])

# Point Transition Matrix (note that this from the perspective of the server)
# Absorbing states: won point, lost point (winner, error)
M_point = np.array([[0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0], 
                        [0, 0, 0, 0, 0, 0, 0], 
                        [p_fh+fserve_boost, p_fh, 0, p_fh, p_fh, 0, 0],
                        [p_bh-fserve_boost, p_bh, 0, p_bh, p_bh, 0, 0],
                        [p_w+fserve_boost, p_w, 0, p_w+fh_boost, p_w - fh_boost, 1, 0],
                        [p_e-fserve_boost, p_e, 1, p_e-fh_boost, p_e + fh_boost, 0, 1]])

res = M_point.dot(init_state)
# stable_state probabilities
for i in range(10000):
  res = M_point.dot(res)
print(res)

[0.      0.      0.      0.      0.      0.62007 0.37993]


In [127]:
#17x17 matrix
p_w = res[5] 
p_l = res[6]
print(p_w)
print(p_l)

#states: 0-0, 15-0, 0-15, 30-0, 0-30, 30-15, 15-30, 40-15, 15-40, D (deuce), AD-IN, AD-OUT, 40-0, 0-40, hold (win), break (lose)

# Game Transition Matrix (note that this from the perspective of the server)
# Absorbing states: serve held, serve broken (won game, lost game)
M_game = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, p_w, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, p_w, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, p_l, 0, p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, p_w, 0, 0, 0, 0, 0, 0, p_l, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, p_l, 0, 0, 0, 0, 0, 0, p_w, 0, 0],
                   [0, 0, 0, 0, 0, 0, p_l, p_w, 0, 0, 0, p_l, p_w, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, p_l, 0, p_w, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, p_w, p_l, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, p_w, 0, 0, p_w, 0, p_w, 0, 1, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, p_l, 0, 0, p_l, 0, p_l, 0, 1]])

init_game = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

res2 = M_game.dot(init_game)
# stable_state probabilities

for i in range(10000):
  res2 = M_game.dot(res2)
print(res2)

0.62007
0.37993000000000005
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.77599741 0.22400259]


In [128]:
'''
Returns probability of winning a point and probability of losing a point.
'''
def simulatePoint():
  #temperature = random number between -0.015 & 0.015 (induces variation)
  tmp = random.randint(-15, 15) / 1000

  p_fserve = 0.675 + tmp
  p_sserve = 0.30 - tmp
  p_df = 0.025
  p_fh = 0.25 - tmp
  p_bh = 0.15 + tmp
  p_w = 0.35 + tmp
  p_e = 0.25 - tmp

  fh_boost = 0.05
  fserve_boost = 0.05
  # 1st serves and forehands are more optimal (offensive/dominant shots)
  # backhands + second serves are more defensive (and more error-prone)

  init_state = np.array([p_fserve, p_sserve, p_df, 0, 0, 0, 0])
  M_point = np.array([[0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0, 0], 
                      [p_fh+fserve_boost, p_fh, 0, p_fh, p_fh, 0, 0],
                      [p_bh-fserve_boost, p_bh, 0, p_bh, p_bh, 0, 0],
                      [p_w+fserve_boost, p_w, 0, p_w+fh_boost, p_w - fh_boost, 1, 0],
                      [p_e-fserve_boost, p_e, 1, p_e-fh_boost, p_e + fh_boost, 0, 1]])

  output = M_point.dot(init_state)
  # stable_state probabilities
  for i in range(10000):
    output = M_point.dot(output)
  return (output[5], output[6])


def getGameMarkov(p_w, p_l):
  M_game = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, p_w, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, p_w, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, p_l, 0, p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, p_w, 0, 0, 0, 0, 0, 0, p_l, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, p_l, 0, 0, 0, 0, 0, 0, p_w, 0, 0],
                    [0, 0, 0, 0, 0, 0, p_l, p_w, 0, 0, 0, p_l, p_w, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, p_l, 0, p_w, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, p_w, p_l, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, p_w, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, p_l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, p_w, 0, 0, p_w, 0, p_w, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, p_l, 0, 0, p_l, 0, p_l, 0, 1]])
  return M_game

def simulateGame():
  p_w, p_l = simulatePoint()
  M_game = getGameMarkov(p_w, p_l)

  # we begin in a state of '0-0'
  init_game = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  output = M_game.dot(init_game)

  # stable_state probabilities
  for i in range(1000):
    # simulate a new point and move towards the result of the game
    p_w, p_l = simulatePoint()
    M_game = getGameMarkov(p_w, p_l)
    output = M_game.dot(output)
  return (output[15], output[16])



def printScores(scores, nameDict):
  print("---------------------------------")
  print(nameDict['p1'], ": ", scores['p1'])
  print(nameDict['p2'], ": ", scores['p2'])
  print("---------------------------------")
  print("\n")



def simulateMatch():
  name1 = input("What is player one's name?: ")
  name2 = input("What is player two's name?: ")

  nameDict = {"p1": name1, "p2": name2}

  scores = {"p1": 0, "p2": 0}
  first = ""
  second = ""
  
  if random.randint(0,1) == 1:
    first = "p1"
    second = "p2"
    print("Based on the result of the coin toss, ", nameDict['p1'], " is to serve first")
    print("\n")
  else:
    first = "p2"
    second = "p1"
    print("Based on the result of the coin toss, ", nameDict['p2'], " is to serve first")

  print("Beginning Match ...")
  # we simulate two services at a time
  while scores['p1'] < 6 and scores['p2'] < 6:
    print(nameDict[first], " to serve:")
    p_held, p_broken = simulateGame()
    if random.random() < p_held:
      scores[first] += 1
      print(nameDict[first], " holds serve!")
    else:
      scores[second] += 1
      print(nameDict[first], " is broken!")
    
    if (scores['p1'] == 6 or scores['p2'] == 6):
      printScores(scores, nameDict)
      break
    
    print(nameDict[second], " to serve:")
    p_held, p_broken = simulateGame()
    if random.random() < p_held:
      scores[second] += 1
      print(nameDict[second], " holds serve!")
    else:
      scores[first] += 1
      print(nameDict[second], " is broken!")

    printScores(scores, nameDict)

    if (scores['p1'] == 6 or scores['p2'] == 6):
      break
  
  if scores['p1'] == scores['p2']:
    print("------ This is roughly a draw! A tiebreak will be required to decide the outcome ------")
  elif scores['p1'] > scores['p2']:
    print("------- ", nameDict['p1'], " has won the match! Stay tune for the post-match coverage! ------")
  else:
    print("------- ", nameDict['p2'], " has won the match! Stay tune for the post-match coverage! ------")
    


print(simulateMatch())


What is player one's name?: Gene Kim
What is player two's name?: Sid
Based on the result of the coin toss,  Sid  is to serve first
Beginning Match ...
Sid  to serve:
Sid  holds serve!
Gene Kim  to serve:
Gene Kim  holds serve!
---------------------------------
Gene Kim :  1
Sid :  1
---------------------------------


Sid  to serve:
Sid  holds serve!
Gene Kim  to serve:
Gene Kim  holds serve!
---------------------------------
Gene Kim :  2
Sid :  2
---------------------------------


Sid  to serve:
Sid  holds serve!
Gene Kim  to serve:
Gene Kim  holds serve!
---------------------------------
Gene Kim :  3
Sid :  3
---------------------------------


Sid  to serve:
Sid  holds serve!
Gene Kim  to serve:
Gene Kim  is broken!
---------------------------------
Gene Kim :  3
Sid :  5
---------------------------------


Sid  to serve:
Sid  is broken!
Gene Kim  to serve:
Gene Kim  holds serve!
---------------------------------
Gene Kim :  5
Sid :  5
---------------------------------


Sid  to 